# Data preparation


In real business scenarios adapting tracking solution, data labeling on video file is required. [The MOTChallenge datasets](https://motchallenge.net/) are the datasets for the task of multiple object tracking, it includes datasets such as MOT15, MOT16, MOT17. In this notebook, we download a video from MOT16 test set to simulate the custom data, and split it into several clips for manual data labeling.

## 1. Prepare raw data

Download MOT16 test data as the raw data. The original data is the test data for [motchallenge MOT16](https://motchallenge.net/sequenceVideos/MOT16-03-raw.webm). To process the video in the following section, we download the `mp4` format data from [FairMOT repo](https://github.com/ifzhang/FairMOT).

In [ ]:
!mkdir datasets
!wget https://raw.githubusercontent.com/ifzhang/FairMOT/master/videos/MOT16-03.mp4 -O datasets/MOT16-03.mp4

### Split video into clips

Install opencv to run data processing.

In [ ]:
!pip install opencv-python

Read a video file and split the video file by each 200 frames. You can change the number of clips by adjusting `clip_interval`, but note that the number of clips cannot be less than 5 as we will separate these clips into train/validation/test dataset for model training.

In [ ]:
import os
import cv2

clip_interval = 200

def split_video(video_path, clip_dir="./datasets/clips"):
    cap = cv2.VideoCapture(video_path)
    
    if not os.path.exists(clip_dir):
        os.makedirs(clip_dir)
    
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    
    frame_cnt = 0
    clip_cnt = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            if frame_cnt % clip_interval == 0:
                if frame_cnt > 0:
                    out.release()
                
                clip_path = f"{clip_dir}/sample_{clip_cnt}.mp4"
                out = cv2.VideoWriter(clip_path, fourcc, fps, (frame_width, frame_height))
                print(f"save clip: {clip_path}")
                clip_cnt += 1
            out.write(frame)
            frame_cnt += 1
        else:
            break
    out.release()

clip_dir = "./datasets/clips"
split_video('./datasets/MOT16-03.mp4', clip_dir=clip_dir)

Upload the clips to S3 Bucket.

In [ ]:
bucket_name = <BUCKET_NAME>
prefix = <PREFIX_NAME>
sample_data_s3uri = f"s3://{bucket_name}/{prefix}/sample-data"

In [ ]:
!aws s3 cp --recursive $clip_dir $sample_data_s3uri

## 2. Label raw data

You can follow the [SageMaker Ground Truth guide](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-getting-started.html) to run the below tasks:
- Step-1: [create a Private workforce(team)](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-create-private-cognito.html)
- Step-2: Add a worker into the private team you created, you will receive an email with the title like `You're invited by aws to work on a labeling project.` which includes `User name`, `Temporary password` and the link for login
- Step-3: Create a labeling job with  as the input datasets in SageMaker Ground Truth Console.
    - Choose `Automated data setup` as **Input data setup**
    - Set `sample_data_s3uri` to **S3 location for input datasets**
    - In **S3 location for output datasets**, choose **Specify a new location** and set the s3 bucket as name such as `s3://{bucket_name}/{prefix}/sample-data-gt`
    - In **IAM Role**, choose a IAM role or create a new one which can access S3 bucket
    - Run **Complete data setup** to complete your input data setup. It will take minutes in this step. You will see `Input data connection successful.` once this step is done.
    - In **Data type**, Choose **Video->Video files**
    - In **Frame extraction**, choose **Use every 5 frame from a video to create a labeling task.**
    - In **Task type->Task category**, choose **Video - Object tracking** and select **Bounding box**
    - Click `next` to go to **Select workers and configure tool**, and choose `Private` in **Worker types**
    - Choose the private team you created before in **Private teams**
    - Leave default values for **Task timeout**
    - In **Video object tracking**, fill in **Task description** with description such as `this is a labelling task video tracking`
    - In **Label values**, add `person` and `car` as label and create labeling job.
    - You will the the status of the labelling job you created is `In progress`
    - Go to **Ground Truth->Labeling workforces** and choose `Private`, you will see `https://xxxxx.labeling.us-east-1.sagemaker.aws` under **Labeling portal sign-in URL** in **Private workforce summary**, and open this link.
    - You can the labeling job you just created before which title is `Track objects across video frames: this is a labelling task video tracking`
    - By clicking **Start working** button, you can start labeling job.
    
- Step-4: Label data by following [the guide](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-video-object-tracking.html). To accelarate the labeling, you can use the **Predict** function to predict the boxes in the current frame.

<img align="center" src="img/label_video.png"></img>


Once finishing a labeling task, you can get the following annotation directory in the defined S3 path.

<div align="center">
    <img width=300 src="img/gt_structure.png">
    <figcaption>Ground Truth Structure</figcaption>
</div>

Under manifest directory, there should be an `out` folder created if we finish labeling all files.
<div align="center">
    <img width=300 src="img/gt_manifest_structure.png">
    <figcaption>Manifest in Ground Truth Structure</figcaption>
</div>

You will see a file `output.manifest` like this:
<div align="center">
    <img width=600 src="img/out_manifest.png">
    <figcaption>output.manifest</figcaption>
</div>

Refer to [Use Amazon SageMaker Ground Truth to Label Data](https://docs.aws.amazon.com/sagemaker/latest/dg/sms.html) for guide of labeling data. You can choose either video files or frame files to label data. 